In [ ]:
import os, shutil            #Operating System
#from pyswmm import Simulation, raingages, Nodes, SystemStats, Links
import pandas as pd
from swmmio.utils.modify_model import replace_inp_section
from swmmio import create_dataframeINP
import swmmio
from swmmio.graphics import swmm_graphics as sg
import math
import numpy as np
import datetime as DT
from functions import *

In [ ]:
# Richiama il solutore SWMM5
def swmm5solver (inpfile):
    if os.path.isfile(inpfile[:-4] + '.rpt'):
        print('Analisi non eseguita: ' + inpfile[:-4] + '.rpt' + ' esiste già')
        return
    
    print('Eeguo l\'analisi di ' + inpfile)
    inpfile = inpfile[:-4] # Nome file senza estensione
    
    if os.sys.platform == 'linux':
        swmmexepath = 'swmm51' ## Per windows bisogna inserire il percorso di swmm5.exe
        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt" "{inpfile}.out" > /dev/null
#        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt"  > /dev/null 2> /dev/null #
        ! rm "{inpfile}.out" # i file out occupano molto  sppazio
        print('Creato file di report ' + inpfile + '.rpt')
    elif os.sys.platform == 'windows':
        #  bisogna aggiungere il percorso di swmm5.exe alla PATH di windows:
        #  https://www.wikihow.com/Run-a-Program-on-Command-Prompt [Metodo2]
        swmmexepath = 'C:\Program Files (x86)\EPA SWMM 5.1.013\swmm5.exe'
        ! {swmmexepath} "{inpfile}.inp" "{inpfile}.rpt" "{inpfile}.out"
        ! del "{inpfile}.out" # i file out occupano molto  sppazio
        print('Creato file di report ' + inpfile + '.rpt')
    else: 
        print('swmm5solver: Sistema operativo non supportato')


In [ ]:
if os.getcwd() != '/home/fanda/Dropbox/doc/trento/Tesi/swmmio':
    os.chdir("./swmmio")
#os.listdir()

file = ! zenity --file-selection  --separator=, --file-filter=*.csv # solo su linux
file = file[0]
file

NewDiameters = pd.read_csv(file)
NewDiameters = NewDiameters.rename(columns={'0' : 'Name','DiametriNuovi': 'Geom1'})
NewDiameters = NewDiameters.set_index('Name')


In [ ]:
# Preambolo

BaseModel = swmmio.Model(r'../inp_file/rovereto_mod.inp')

swmmio.Model.subcatchments

# Crea i dataframe con le informazioni dell'inpfile originale
## RainGages
GAGES = create_dataframeINP(BaseModel.inp.path,'[RAINGAGES]')

## TimeSeries
TIMESERIES = create_dataframeINP(BaseModel.inp.path,'[TIMESERIES]')
TIMESERIES = pd.DataFrame( {'[TIMESERIES]' : TIMESERIES['[TIMESERIES]'].unique()} ) # Elimino i doppi
TIMESERIES = pd.DataFrame(TIMESERIES['[TIMESERIES]'].str.split().tolist(),
                          columns=['Name', 'Date', 'Time'],
                         )
TIMESERIES = TIMESERIES.set_index('Name')

## Nodi Dividers
DIVIDERS = create_dataframeINP(BaseModel.inp.path,'[DIVIDERS]')
DIVIDERS = pd.DataFrame(DIVIDERS['[DIVIDERS]'].str.split().tolist(), 
                        columns = ['Name','Elevation', 'DivertedLink', 
                                   'Type', 'CutoffFlow','MaxDepth', 
                                   'P1', 'P2', 'P3'], 
                                    )
DIVIDERS = DIVIDERS.set_index('Name')

NODES = BaseModel.nodes()
SUBCATCHMENTS = BaseModel.subcatchments()
CONDS = BaseModel.conduits()

# Definizione dei diametri commericiali
# https://www.oppo.it/tabelle/diametri_interni_tubi.htm#fibrocemento
DiametriCommericali = [.060,.080,.100,.125,
                       .150,.170,.200,.225,
                       .250,.300,.350,.400,
                       .450,.500,.600,.700,
                       .800,.900,1.00, 1.1,
                        1.2,1.3,1.4,1.5,1.6]



In [ ]:
inpfilepath = BaseModel.inp.path


for iteration in [1,2]: # 6 iterazioni
    
    print('\nIterazione N: ' + str(iteration) + '\n')
    
    try: 
        os.mkdir('../swmmio/iter' + str(iteration) )
    except FileExistsError:
        pass
    
    # Inizializzo dataframe con i risultati delle analisi
    df_MaxQ = pd.DataFrame()
    df_MaxHr = pd.DataFrame()
    df_MaxDay = pd.DataFrame()
    df_MaxV = pd.DataFrame()

    # Verifica della Rete
    ## Per TR 10 Anni
    for timeserie in TIMESERIES[TIMESERIES.index.str.contains('TR10_')].index: 
    #for timeserie in TIMESERIES.index:
        GAGES.DataSourceName = timeserie
        newfilepath = os.path.join('../swmmio/iter' + str(iteration) + '/'+ \
                                   BaseModel.inp.name + '_' + timeserie  + '.inp')

        newfilebase = newfilepath[:-4] # Nome file senza estensione
        print(newfilepath)

        # Creo nuovo inp file con un con un diversa timeseries in RAINGAGE
        shutil.copyfile(inpfilepath, newfilepath)
        replace_inp_section(newfilepath, '[RAINGAGES]', GAGES)

        # ESEGUO L'ANALISI Con SWMM5
        swmm5solver(newfilepath)
        
        mymodel = swmmio.Model(newfilepath)
        nodes = NODES
        conds = mymodel.conduits()

        # Plotto il riempimento delle Condotte e dei Nodi
        print('plot del grafico ', newfilebase + '_riempimento.png')
        plotModel(inpfilepath = newfilepath, 
                  title = timeserie, 
                  annotation = 'Percentuale di riempimento nelle condotte',
                  outfile = newfilebase + '_riempimento.png',
                  #Property_nodes = 'MaxNodeDepthReported',
                  Property_nodes = '',
                  Property_conds = 'MaxDPerc',
                 )
        
        # inserisco i risultati delle analisi nei dataframe risultati
        df_MaxQ[str(timeserie)] = conds.MaxQ
        df_MaxHr[str(timeserie)] = conds.MaxHr
        df_MaxDay[str(timeserie)] = conds.MaxDay
        df_MaxV[str(timeserie)] = conds.MaxV

    df = pd.DataFrame()
    
    dfConds = pd.DataFrame()
    dfConds['Shape'] = conds.Shape
    dfConds['Geom1'] = NewDiameters.Geom1.tolist()
    dfConds['Geom2'] = conds.Geom2
    dfConds['Geom3'] = conds.Geom3
    dfConds['Geom4'] = conds.Geom4
    dfConds['Barrels'] = conds.Barrels
    
    print(dfConds)

    # Creo nuovo file di progetto con le nuove dimensioni
    
    newinpfilepath = BaseModel.inp.path[:-4] + '_iter' + str(iteration) + '.inp'
    
    shutil.copyfile(inpfilepath, newinpfilepath)
    replace_inp_section(newinpfilepath, '[XSECTIONS]', dfConds)

    # Plot della Dimensione delle condotte
    plotGeomNetwork(newinpfilepath)
    
    # Quante linee sono cambiate nel file inp?
    diff = ! diff {inpfilepath} {newinpfilepath} | wc -l
    
    print('Numero approssimativo condotte cambiate ~' + str(diff))
    
    if diff == 0:
        print('i file ' + \
             str(inpfilepath) + \
             ' e ' + \
             str(newinpfilepath) + \
             ' sono uguali!')
        break
    
    inpfilepath = newinpfilepath


In [ ]:
C_new = (conds.Geom1.apply(Cp, H = 4) * conds.Length)[CONDS.Length < 399].sum() + NODES.MaxDepth.apply(Cm).sum()
C_old = (CONDS.Geom1.apply(Cp, H = 4) * CONDS.Length)[CONDS.Length < 399].sum()+ NODES.MaxDepth.apply(Cm).sum()

print('costo per la nuova rete =', C_new)
print('costo per la vecchia rete =', C_old)

In [ ]:
CONDS

In [ ]:
NODES.InvertElev.min()

In [ ]:
NODES.InvertElev.max()

In [ ]:
SUBCATCHMENTS